In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import os
import pandas as pd
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

from sklearn.utils.class_weight import compute_class_weight

from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-07-26 20:40:35.745061: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/azzanf/Documents/Azza/TA/Code/.conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/azzanf/Documents/Azza/TA/Code/.conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['dlopen(/Users/azzanf/Documents/Azza/TA/Code/.conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so, 0x0006): symbol not found in flat namespace (__ZN3tsl2io7DirnameENSt3__117basic_string_viewIcNS1_11char_traitsIcEEEE)']
  warnings.warn(f"unabl

In [ ]:
def load_images(df):
    image_data = []
    
    for index, row in df.iterrows():
        image_path = row['filepath']
        category = row['Category']
        image = tf.keras.preprocessing.image.load_img(image_path, target_size=(64, 64), color_mode='grayscale')
        image = tf.keras.preprocessing.image.img_to_array(image)
        image_data.append(image)

    image_data = np.array(image_data)
    image_data = image_data.astype('float32') / 255.0
    
    return image_data

def load_augmented_images(df, batch_size=32):
    datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1, 
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    image_data = []
    label_data = []
    
    for index, row in df.iterrows():
        image_path = row['filepath']
        category = row['Category']
        image = tf.keras.preprocessing.image.load_img(image_path, target_size=(64, 64), color_mode='grayscale')
        image = tf.keras.preprocessing.image.img_to_array(image)
        
        # Apply data augmentation individually to each image
        augmented_images = datagen.flow(np.expand_dims(image, axis=0), batch_size=batch_size, shuffle=False)
        for _ in range(batch_size):
            augmented_image = augmented_images.next()
            image_data.append(augmented_image[0])
            label_data.append(category)
    
    # Convert the lists to numpy arrays
    image_data = np.array(image_data)
    label_data = np.array(label_data)
    
    return image_data, label_data

In [ ]:
# Load train_data.csv
train_df = pd.read_csv('train_data.csv')
val_df = pd.read_csv('val_data.csv')
test_df = pd.read_csv('test_data.csv')

In [ ]:
name_classes = ["Drowsy", "Neutral"]
num_classes = len(name_classes)

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(train_df['Category'])

LabelEncoder()

In [ ]:
train_images, train_labels = load_augmented_images(train_df)
val_images, val_labels = load_augmented_images(val_df)
test_images = load_images(test_df)

train_labels = label_encoder.transform(train_labels)
val_labels = label_encoder.transform(val_labels)
test_labels = label_encoder.transform(test_df['Category'])

train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
val_labels = tf.keras.utils.to_categorical(val_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

In [ ]:
# Define the CNN architecture
def create_cnn_model(num_classes):
    model = models.Sequential()
    
    # First set of Convolutional, BatchNormalization, and ReLU layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Second set of Convolutional, BatchNormalization, and ReLU layers
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Third set of Convolutional, BatchNormalization, and ReLU layers
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Flatten the output to prepare for dense layers
    model.add(layers.Flatten())
    
    # Dense layers
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    
    # Output layer with units corresponding to the number of classes
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

In [ ]:
model = create_cnn_model(num_classes)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [ ]:
class_weights = compute_class_weight('balanced', np.unique(train_labels.argmax(axis=1)), train_labels.argmax(axis=1))
class_weights = dict(enumerate(class_weights))

In [ ]:
# Train the model
model.fit(train_images, train_labels, epochs=100, batch_size=64, validation_data=(val_images, val_labels), callbacks=[early_stop], class_weight=class_weights)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

Epoch 1/100
244/244 [==============================] - 8s 28ms/step - loss: 0.4778 - accuracy: 0.8127 - val_loss: 0.7793 - val_accuracy: 0.4988
Epoch 2/100
244/244 [==============================] - 7s 27ms/step - loss: 0.0701 - accuracy: 0.9748 - val_loss: 0.2472 - val_accuracy: 0.8859
Epoch 3/100
244/244 [==============================] - 7s 27ms/step - loss: 0.0479 - accuracy: 0.9847 - val_loss: 0.1159 - val_accuracy: 0.9530
Epoch 4/100
244/244 [==============================] - 7s 27ms/step - loss: 0.0278 - accuracy: 0.9908 - val_loss: 0.0789 - val_accuracy: 0.9754
Epoch 5/100
244/244 [==============================] - 7s 27ms/step - loss: 0.0249 - accuracy: 0.9919 - val_loss: 0.2921 - val_accuracy: 0.8744
Epoch 6/100
244/244 [==============================] - 7s 27ms/step - loss: 0.0203 - accuracy: 0.9927 - val_loss: 0.0149 - val_accuracy: 0.9964
Epoch 7/100
244/244 [==============================] - 7s 27ms/step - loss: 0.0163 - accuracy: 0.9944 - val_loss: 0.1337 - val_accuracy:

In [ ]:
# Convert the Keras model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# # Save the TensorFlow Lite model to a file
# with open('model_cnn.tflite', 'wb') as f:
#     f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp_ijn0xb6/assets


INFO:tensorflow:Assets written to: /tmp/tmp_ijn0xb6/assets
2023-07-26 10:57:56.914525: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-07-26 10:57:56.914566: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
